# Install and Import Dependencies

In [3]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
import threading
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Creating functions

In [2]:
def calculate_angle(a,b,c):
    # Calculates the angle between three points: a, b, and c.
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def start_detection():
    # Starts the pose detection thread.
    global detection_thread, stop_detection_flag
    stop_detection_flag = False
    detection_thread = threading.Thread(target=perform_detection)
    detection_thread.start()

def stop_detection():
    # Stops the pose detection.
    global stop_detection_flag
    stop_detection_flag = True

# Determining the joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# Display the Output

In [9]:
cap = cv2.VideoCapture(0)

# variable/s
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            shoulder_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y], 
                                             shoulder, elbow)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(round(shoulder_angle, 2)), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )
            

            if 90 <= shoulder_angle and shoulder_angle <= 180:
                if 165 < angle <= 180:
                    stage = "Not throwing"
                else:
                    stage = "Throwing"
            else:
                stage = None
                
            # Setup status box
            cv2.rectangle(image, (0,0), (450,130), (255,255,255), -1)

            # Visualize stage
            cv2.putText(image, stage, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)

                       
        except:
            pass
       
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Detecting Both Hands

In [7]:
cap = cv2.VideoCapture(0)

   
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angles
            right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_shoulder_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y], 
                                                   right_shoulder, right_elbow)
            
            left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_shoulder_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], 
                                                  left_shoulder, left_elbow)
            
            # Visualize angles
            cv2.putText(image, "Right Angle: " + str(round(right_angle, 2)), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            cv2.putText(image, "Right Shoulder Angle: " + str(round(right_shoulder_angle, 2)), 
                           tuple(np.multiply(right_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            cv2.putText(image, "Left Angle: " + str(round(left_angle, 2)), 
                           tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            cv2.putText(image, "Left Shoulder Angle: " + str(round(left_shoulder_angle, 2)), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Determine stage for right hand
            if 90 <= right_shoulder_angle <= 180:
                if 165 < right_angle <= 180:
                    right_stage = "Not throwing"
                else:
                    right_stage = "Throwing"
            else:
                right_stage = None

            # Determine stage for left hand
            if 90 <= left_shoulder_angle <= 180:
                if 165 < left_angle <= 180:
                    left_stage = "Not throwing"
                else:
                    left_stage = "Throwing"
            else:
                left_stage = None
            
            # Setup status box
            cv2.rectangle(image, (0,0), (450,130), (255,255,255), -1)
            
            # Visualize stages
            cv2.putText(image, "Right Hand: " + str(right_stage), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, "Left Hand: " + str(left_stage), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

                       
        except:
            pass
       
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Inserting a User GUI

In [15]:
# Using start_detection(): and stop_detection():

def perform_detection():
    cap = cv2.VideoCapture(0)
    stage = None

    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
          
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                # Calculate angles
                right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                right_shoulder_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y], 
                                                       right_shoulder, right_elbow)
                
                left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                left_shoulder_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], 
                                                      left_shoulder, left_elbow)
                
                # Visualize angles
                cv2.putText(image, "Right Angle: " + str(round(right_angle, 2)), 
                               tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                cv2.putText(image, "Right Shoulder Angle: " + str(round(right_shoulder_angle, 2)), 
                               tuple(np.multiply(right_shoulder, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                cv2.putText(image, "Left Angle: " + str(round(left_angle, 2)), 
                               tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                cv2.putText(image, "Left Shoulder Angle: " + str(round(left_shoulder_angle, 2)), 
                               tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                # Determine stage for right hand
                if 90 <= right_shoulder_angle <= 180:
                    if 165 < right_angle <= 180:
                        right_stage = "Not throwing"
                    else:
                        right_stage = "Throwing"
                else:
                    right_stage = None
    
                # Determine stage for left hand
                if 90 <= left_shoulder_angle <= 180:
                    if 165 < left_angle <= 180:
                        left_stage = "Not throwing"
                    else:
                        left_stage = "Throwing"
                else:
                    left_stage = None
                
                # Setup status box
                cv2.rectangle(image, (0,0), (450,130), (255,255,255), -1)
                
                # Visualize stages
                cv2.putText(image, "Right Hand: " + str(right_stage), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(image, "Left Hand: " + str(left_stage), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    
                           
            except:
                pass
           
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )               
            cv2.imshow('Mediapipe Feed', image)
    
            if cv2.waitKey(10) & 0xFF == ord('q') or stop_detection_flag:
                break
    
        cap.release()
        cv2.destroyAllWindows()

# Create the Tkinter window
window = tk.Tk()
window.title("Cricket Bowling Illigal Action Detection")
window.geometry("500x300")

# Create a button to start the detection
start_button = tk.Button(window, text="Start Detection", command=start_detection, height=2, width=20)
start_button.pack(pady=50)

# Create a button to stop the detection
stop_button = tk.Button(window, text="Stop Detection", command=stop_detection, height=2, width=20)
stop_button.pack(pady=10)

# Initialize stop detection flag
stop_detection_flag = False

# Start the Tkinter event loop
window.mainloop()

# Full Code (For running the code at once)

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
import threading

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def start_detection():
    global detection_thread, stop_detection_flag
    stop_detection_flag = False
    detection_thread = threading.Thread(target=perform_detection)
    detection_thread.start()

def stop_detection():
    global stop_detection_flag
    stop_detection_flag = True

def perform_detection():
    cap = cv2.VideoCapture(0)
    stage = None

    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
          
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                # Calculate angles
                right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                right_shoulder_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y], 
                                                       right_shoulder, right_elbow)
                
                left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                left_shoulder_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], 
                                                      left_shoulder, left_elbow)
                
                # Visualize angles
                cv2.putText(image, "Right Angle: " + str(round(right_angle, 2)), 
                               tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                cv2.putText(image, "Right Shoulder Angle: " + str(round(right_shoulder_angle, 2)), 
                               tuple(np.multiply(right_shoulder, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                cv2.putText(image, "Left Angle: " + str(round(left_angle, 2)), 
                               tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                cv2.putText(image, "Left Shoulder Angle: " + str(round(left_shoulder_angle, 2)), 
                               tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                
                # Determine stage for right hand
                if 90 <= right_shoulder_angle <= 180:
                    if 165 < right_angle <= 180:
                        right_stage = "Not throwing"
                    else:
                        right_stage = "Throwing"
                else:
                    right_stage = None
    
                # Determine stage for left hand
                if 90 <= left_shoulder_angle <= 180:
                    if 165 < left_angle <= 180:
                        left_stage = "Not throwing"
                    else:
                        left_stage = "Throwing"
                else:
                    left_stage = None
                
                # Setup status box
                cv2.rectangle(image, (0,0), (450,130), (255,255,255), -1)
                
                # Visualize stages
                cv2.putText(image, "Right Hand: " + str(right_stage), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(image, "Left Hand: " + str(left_stage), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    
                           
            except:
                pass
           
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )               
            cv2.imshow('Mediapipe Feed', image)
    
            if cv2.waitKey(10) & 0xFF == ord('q') or stop_detection_flag:
                break
    
        cap.release()
        cv2.destroyAllWindows()

# Create the Tkinter window
window = tk.Tk()
window.title("Bowling Illigal Action Detection")
window.geometry("500x300")

# Create a button to start the detection
start_button = tk.Button(window, text="Start Detection", command=start_detection, height=2, width=20)
start_button.pack(pady=50)

# Create a button to stop the detection
stop_button = tk.Button(window, text="Stop Detection", command=stop_detection, height=2, width=20)
stop_button.pack(pady=10)

# Initialize stop detection flag
stop_detection_flag = False

# Start the Tkinter event loop
window.mainloop()